In [ ]:
import sims
import numpy as np
import msprime
import pandas as pd

import useful
import seaborn
import matplotlib
from matplotlib import pyplot as plt
from importlib import reload


import json

In [ ]:
# Generation time, mutation rate and recomination rate
RR = 1e-8
MU = 1.29e-8 
GEN_time = 29.0 

# Split Times
T_NEAND_migration = 55000 #time of Neanderthal migration into Out_of_africa population
T_NEAND_AMH = 550000 # split time between AMH and Neanderthal
T_OOF_AF = 65700 # Out_of_Africa migration time
T_NEAND_samples = 38000

# Effective population size
N_ANC = 18500 # N_e of common  AMH and NEanderthal population 
N_ND = 3400 # N_e of Neanderthal
N_AMH = 23000 # N_e of AMH
N_OOF = 1861 # N_e of Out_of_Africa population
N_AF = 27600 # N_e of Africans
N_EU = 13377 #N_e of Europeans

N_EU_bottleneck = 1080
N_EU_growth = 1450
T_EU_growth = 31900
gr_rate = 0.00202
Portion_admix = 0.03

len_sequence = 1e7 # DNA sequence length

n = 250 # number of generated   AF samples
n_neand = 10 #number of generated Neanderthals

rand_sd =1234 #random seed
T = np.array([T_NEAND_migration, T_NEAND_AMH, T_OOF_AF])/GEN_time


N_ND = 3400 # N_e of Neanderthal
N_e = np.array([N_ANC, N_ND, N_AMH, N_OOF, N_AF, N_EU])

n_eu=1
ploidy=2
L=1000
N_ref_pop=250
N_neanderthal=6

In [ ]:
number_chr=2
ts_mas, ND_true_tracts_mas = sims.sim_history_archaic_many_ts(GEN_time, len_sequence, RR, MU, N_e, T,  n, rand_sd, n_neand,  
                              T_NEAND_samples/GEN_time, n_eu, N_EU_growth, 
                              T_EU_growth/GEN_time, N_EU_bottleneck, gr_rate, Portion_admix, ploidy, number_chr)

In [ ]:
#functions that move the j-th set of tracts by j*len_sequence
def many_in_one(nd_tracts_mas, number_chr):
    nd2=[]
    for _ in range(len(ts_mas)):
        for j in range(len(nd_tracts_mas[_])):
            nd2.append([nd_tracts_mas[_][j][0]+_*len_sequence, nd_tracts_mas[_][j][1]+_*len_sequence])

    return [useful.tracts_eu(nd2, len_sequence*len(ts_mas) ), nd2]

In [ ]:
true_nd=[ND_true_tracts_mas[_][0] for _ in range(len(ts_mas))]  
real_tracts_in_states = many_in_one(true_nd,number_chr)

# DAIseg preparations

In [ ]:
for _ in range(len(ts_mas)):
    sims.create_obs_txt('obs.chr'+str(_+1)+'.txt', ploidy,  n_eu, ts_mas[_], N_ref_pop, N_neanderthal, n)
sims.create_bed_smpls_arch_cov('samples.txt', 'test.bed','arch.cover.txt', len_sequence, 0.9999, 1, n_eu, ploidy)

In [ ]:
! ./run.daiseg.long.sh

In [ ]:
daiseg=[]
for _ in range(number_chr):
    daiseg.append(useful.read_out('out.chr'+str(_+1)+'.archaic.txt')[0])


tracts_daiseg=many_in_one(daiseg, number_chr)
df_daiseg=sims.df_result_lonf_chr(real_tracts_in_states, tracts_daiseg, 6,  250, 2)

# HMMIX preparations

In [ ]:
#preparation_to_hmmix (ancestral alleles, fasta, vcf)

j_son={"ingroup":['tsk_'+str(_) for _ in range(0, n_eu)], "outgroup" : ['tsk_'+str(_) for _ in range(n_eu, n_eu+N_ref_pop)]}
with open('./hmmix/individuals.json', 'w') as f:
    json.dump(j_son, f)

for _ in range(len(ts_mas)):
    with open('./hmmix/ts.'+'chr'+str(_+1)+'.vcf', "w") as vcf_file:
        ts_mas[_].write_vcf(vcf_file, contig_id=str(_+1))

sims.make_ancestral_fasta('./hmmix/ancestral', ts_mas, len_sequence)

In [ ]:
#then run ./hmmix.sh from hmmix directory

In [ ]:
file='./hmmix/out.tsk_0.hap1.txt'
hmmix_nd=sims.read_noND(file, 0.9,2)
#The goal of this cell in to make ONE big chr
tracts_hmmix=many_in_one(hmmix_nd, number_chr)
df_hmmix = sims.df_result_lonf_chr(real_tracts_in_states, tracts_hmmix, 6,  250, 2)        